<a href="https://colab.research.google.com/github/ManuSanchez02/7506R-2c2022-GRUPO09/blob/main/tp2/7506R_TP2_GRUPO09_ENTREGA_N1(nlp).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Configuración inicial

Importamos las bibliotecas necesarias para el análisis.

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

from sklearn import metrics
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV
import regex

!pip install xgboost

np.random.seed(1)
SEMILLA = 1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Funciones útiles.

Definimos funciones que se van a utilizar.

In [2]:
def imprimir_metricas_regresion(target_test, precios_predichos):
    # Error cuadrático medio
    mse = metrics.mean_squared_error(
        y_true=target_test, y_pred=precios_predichos, squared=True
    )

    print(f"El error (mse) de test es: {mse}")

    # Raíz del error cuadrático medio
    rmse = metrics.mean_squared_error(
        y_true=target_test, y_pred=precios_predichos, squared=False
    )

    print(f"El error (rmse) de test es: {rmse}")

    # R2 score
    r2 = metrics.r2_score(target_test, precios_predichos)
    print(f"El score R2 es: {r2}")


In [3]:
columnas = ["property_price", "id", "property_description"]

def build_target_and_dataset(dataset):
  target = dataset["property_price"]
  dataset.drop(columns=columnas, inplace=True)
  dataset = pd.get_dummies(dataset, drop_first=True)
  return target, dataset

Importamos los datasets ya preprocesados.

In [4]:
dataset_train = pd.read_csv("https://github.com/ManuSanchez02/7506R-2c2022-GRUPO09/blob/main/tp2/datasets/dataset_train_preprocesado.csv?raw=True")
dataset_test = pd.read_csv("https://github.com/ManuSanchez02/7506R-2c2022-GRUPO09/blob/main/tp2/datasets/datasetc_test_preprocesado.csv?raw=True")

dataset_train.drop(["Unnamed: 0"], axis = 1, inplace=True)
dataset_test.drop(["Unnamed: 0"], axis = 1, inplace=True)

In [5]:
dataset_pln_train = dataset_train.copy()
dataset_pln_test = dataset_test.copy()

Importamos y hacemos un head del dataset con las descripciones para ver qué forma tiene.

In [8]:
dataset_descrip = pd.read_csv("/content/drive/MyDrive/OrgaTP2/properati_argentina_2021_decrip.csv")
dataset_descrip.head()

,id,property_description
0,BZCeiGkJr1WBUi6pKJQwJQ==,Corredor Responsable: MARIA ALEJANDRA GENOVEVA...
1,zuHOGgUE0UB71W7m/YCLoA==,¡HERMOSO PH TIPO CASA DE 2 AMBIENTES!<br>APTO ...
2,TUL813tXchVBlHUsfaA6DA==,Excelente PH en Núñez 4 amb!!!! Oportunidad !!...
3,gn55JPmcjftBV/at1a7fPg==,"UNICO PH DE 4 DORMITORIOS EN IMPECABLE ESTADO,..."
4,qg4m3QHcuo1+wMmyp7kx3Q==,Ph 2 Amb. Villa Crespo exclusivo uso profesion...


Unimos el dataset preprocesado con el que contiene las descripciones.

In [9]:
dataset_pln_train = dataset_pln_train.merge(dataset_descrip, left_on='id', right_on='id')
dataset_pln_test = dataset_pln_test.merge(dataset_descrip, left_on='id', right_on='id')
dataset_pln_train.head()

,id,latitud,longitud,barrio,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,property_description
0,/KS9ys5/j+tOi7ZtBH0beg==,-34.554271,-58.457668,Belgrano,Departamento,2,1,59,59,178000,Hermoso departamento 2 ambientes piso alto a e...
1,F5QJsmufcrReRAHbc84lxQ==,-34.613544,-58.368072,Monserrat,Departamento,3,2,71,71,129000,Corredor Responsable: Cristian Arnal Ponti - C...
2,MJP64X970Gj+Zdvn592RWA==,-34.571020,-58.433125,Palermo,Departamento,2,1,79,59,249000,Corredor Responsable: Ariel Champanier - CUCIC...
3,SEUBJTexychILiuTsn9L3Q==,-34.597384,-58.405139,Barrio Norte,Departamento,4,3,91,85,165000,SUPER OPORTUNIDAD! AL MEJOR VALOR DE M2 DEL M...
4,gQdKfHVQXAfnza6dQte4ng==,-34.591646,-58.418064,Palermo,Departamento,1,1,37,32,105000,Corredor Responsable: Hernan Perrone CUCICBA 2...


Normalizamos las descripciones para que sea más fácil realizar el análisis.

In [10]:
def normalizar(descripcion):
  descripcion = descripcion.lower()
  descripcion = descripcion.replace('á', 'a')
  descripcion = descripcion.replace('é', 'e')
  descripcion = descripcion.replace('í', 'i')
  descripcion = descripcion.replace('ó', 'o')
  descripcion = descripcion.replace('ú', 'u')
  return descripcion

Definimos una lista de palabras que no son relevantes a la hora de elegir aspectos, y creamos un diccionario que contiene todas las palabras de las descripciones con sus frecuencias, para ver cuáles podemos tomar como aspectos.

In [12]:
ignorar = ["a", "ante", "cabe", "con", "de", "desde", "durante", "en", 
           "entre", "hacia", "hasta", "mediante", "para", "por", "según",
           "sin", "so", "sobre", "tras", "versus", "via", "el", "la", "los", 
           "del", "las", "los", "etc", "es", "era", "br", "y", "que", "al", 
           "o", "un", "ba", "una", "se", "", "cocina", "ambientes", "piso", 
           "pisos", "propiedad", "son", "muy", "esta", "dos", "cuadras",
           "completo", "corredor", "responsable", "inmueble", "excelente", 
           "amplio", "accede", "cucicba", "medidas", "cuenta", "lendar", 
           "queres", "ley", "podes", "gran", "aire", "simula", "metros",
           "mls", "personas", "parte", "inmobiliario", "encuentra", "independiente",
           "todas", "todos", "unidades", "presente", "tiene", "este", "mas",
           "unidad", "aviso", "operacion", "bajo", "propietario", "como", "informacion",
           "operaciones", "mts", "discapacidades"
           ]

In [13]:
def obtener_aspectos(dataset):
  cantidades = {}
  for descripcion in dataset['property_description']:
    descripcion = normalizar(descripcion)
    palabras = re.split('[^a-z]', descripcion)
    for palabra in palabras:
      if not palabra in ignorar and len(palabra) > 2:
        cantidades[palabra] = cantidades.get(palabra, 0) + 1
  return cantidades

diccionario_aspectos = obtener_aspectos(dataset_pln_train)

Tomamos las 50 palabras más frecuentes como aspectos relevantes de las descripciones, y buscamos frases que las contengan para considerar también como aspectos.

In [14]:
aspectos_comunes = sorted(diccionario_aspectos.items(), key=lambda x: x[1], reverse=True)[:50]
for aspecto in aspectos_comunes:
  print(aspecto)

('balcon', 65617)
('comedor', 60511)
('living', 58609)
('departamento', 55953)
('edificio', 51346)
('dormitorio', 44593)
('expensas', 42030)
('frente', 41990)
('valor', 40105)
('placard', 32240)
('casa', 29942)
('luminoso', 29466)
('venta', 29335)
('lavadero', 28301)
('dormitorios', 27703)
('prestamo', 26319)
('terraza', 26252)
('cuota', 25832)
('contacto', 25406)
('ubicacion', 24576)
('vista', 23644)
('salida', 23598)
('espacio', 23272)
('cochera', 22612)
('compra', 22369)
('zona', 21992)
('patio', 20780)
('ubicado', 20579)
('linea', 20472)
('barrio', 20428)
('planta', 20358)
('mesada', 19960)
('principal', 19285)
('acceso', 19023)
('parrilla', 18774)
('accesible', 18627)
('subte', 18562)
('servicios', 18426)
('estado', 17936)
('servicio', 17562)
('acondicionado', 17071)
('agua', 16941)
('toilette', 16738)
('hall', 15808)
('comercial', 15729)
('estacion', 15541)
('calidad', 15113)
('madera', 14985)
('suite', 14645)
('calefaccion', 14427)


In [15]:
aspectos_comunes_lista = [aspecto[0] for aspecto in aspectos_comunes]
aspetos_regex = '|'.join(aspectos_comunes_lista)

def obtener_matches(dataset, regex_match, cantidad):
  cantidades = {}
  for descripcion in dataset['property_description']:
    descripcion = normalizar(descripcion)
    frases = re.findall(regex_match, descripcion)
    for frase in frases:
          cantidades[frase] = cantidades.get(frase, 0) + 1
  
  return dict(sorted(cantidades.items(), key=lambda item: item[1], reverse=True)[:cantidad])

grupo_aspectos = f"(?:{aspetos_regex})"
diccionario_frases = {}
diccionario_frases.update(obtener_matches(dataset_pln_train, "con " + grupo_aspectos + " \w{4,}", 10))
diccionario_frases.update(obtener_matches(dataset_pln_train, "\w{4,} con " +  grupo_aspectos, 10))
diccionario_frases.update(obtener_matches(dataset_pln_train, "\w{4,} " +  grupo_aspectos, 10))
# diccionario_frases.update(obtener_matches(dataset_pln_train, "\w{4,} de " +  grupo_aspectos, 10))


In [16]:
frases_comunes = sorted(diccionario_frases.items(), key=lambda x: x[1], reverse=True)
for frase in frases_comunes:
  print(frase)

('living comedor', 23655)
('aire acondicionado', 16951)
('bajo mesada', 11551)
('excelente ubicacion', 9930)
('dormitorio principal', 7251)
('amplio living', 6901)
('comedor con salida', 5648)
('buen estado', 5578)
('excelente estado', 5318)
('primera calidad', 5278)
('super luminoso', 5267)
('dormitorio con placard', 4553)
('con vista abierta', 3667)
('con espacio para', 3025)
('ambientes con balcon', 2979)
('con comedor diario', 2794)
('dormitorios con placard', 2671)
('frente con balcon', 2394)
('cocina con comedor', 1934)
('terraza con parrilla', 1901)
('con placard empotrado', 1873)
('con balcon corrido', 1546)
('ambientes con cochera', 1369)
('cocina con lavadero', 1351)
('balcon con vista', 1303)
('con lavadero incorporado', 1019)
('con balcon frances', 957)
('con living comedor', 902)
('con vista panoramica', 847)
('con cochera fija', 782)


Una vez generada la lista de aspectos (palabras + frases) los agregamos al dataset como columnas nuevas.

In [17]:
def agregar_aspectos(dataset, aspectos):
  for aspecto in aspectos:
    dataset[aspecto] = dataset["property_description"].apply(contiene_aspecto, args=(aspecto,))

def contiene_aspecto(descripcion, aspecto):
  descripcion = normalizar(descripcion)
  return int(None != re.search(aspecto, descripcion))

frases_comunes_lista = [frase[0] for frase in frases_comunes]
aspectos_comunes_lista = [aspecto[0] for aspecto in aspectos_comunes]
agregar_aspectos(dataset_pln_train, frases_comunes_lista)
agregar_aspectos(dataset_pln_train, aspectos_comunes_lista)
agregar_aspectos(dataset_pln_test, frases_comunes_lista)
agregar_aspectos(dataset_pln_test, aspectos_comunes_lista)

dataset_pln_train.head()

,id,latitud,longitud,barrio,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,...,acondicionado,agua,toilette,hall,comercial,estacion,calidad,madera,suite,calefaccion
0,/KS9ys5/j+tOi7ZtBH0beg==,-34.554271,-58.457668,1,Departamento,2,1,59,59,178000,...,0,0,0,0,0,0,0,0,0,1
1,F5QJsmufcrReRAHbc84lxQ==,-34.613544,-58.368072,1,Departamento,3,2,71,71,129000,...,0,0,0,0,0,0,0,0,0,0
2,MJP64X970Gj+Zdvn592RWA==,-34.571020,-58.433125,0,Departamento,2,1,79,59,249000,...,0,0,1,1,0,0,0,0,1,1
3,SEUBJTexychILiuTsn9L3Q==,-34.597384,-58.405139,1,Departamento,4,3,91,85,165000,...,0,0,0,0,0,0,0,0,0,0
4,gQdKfHVQXAfnza6dQte4ng==,-34.591646,-58.418064,0,Departamento,1,1,37,32,105000,...,0,0,0,0,0,0,0,0,0,0


## XGBoost

Entrenamos dos modelos XGBoost: uno utilizando los hiperparámetros obtenidos en el primer trabajo y otro con hiperparámetros optimizados para el nuevo dataset. 

La idea es generar modelos de regresión que puedan predecir el precio de la propiedad.

Primero construimos el target y adaptamos el dataset.

In [18]:
(target_train, dataset_train) = build_target_and_dataset(dataset_pln_train)
(target_test, dataset_test) = build_target_and_dataset(dataset_pln_test)

### Parámetros del TP1

Entrenamos al primer modelo e imprimimos sus métricas de test y de train.

In [19]:
xgb = XGBRegressor(n_estimators= 500, min_child_weight= 6, max_depth= 12, learning_rate= 0.07)
model = xgb.fit(dataset_train, target_train)

[19:38:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Primero vemos cómo es la performance con los datos de train. Vemos que performa demasiado bien, por lo que podría haber overfitting.

In [20]:
precios_predichos_xgb_train = model.predict(dataset_train)
imprimir_metricas_regresion(target_train, precios_predichos_xgb_train)

El error (mse) de test es: 59773704.05668307
El error (rmse) de test es: 7731.345552792416
El score R2 es: 0.9944272525801775


Ahora analizamos con los datos de test.

In [21]:
precios_predichos_xgb = model.predict(dataset_test)
imprimir_metricas_regresion(target_test, precios_predichos_xgb)

El error (mse) de test es: 955039406.1595
El error (rmse) de test es: 30903.711850835978
El score R2 es: 0.9100983586260344


Las métricas también son muy buenas, por lo que si bien puede haber un poco de overfitting el modelo está prediciendo bien los resultados. 

En el TP1 los resultados habían sido: 
  - El error (mse) de test es: 1194394914.1879835
  - El error (rmse) de test es: 34560.0190131311
  - El score R2 es: 0.8875645071859526

por lo que vemos una mejora. Sin embargo, buscamos un nuevo modelo con los hiperparámetros optimizados a ver si mejoran aún más los resultados.

### Nuevos hiperparámetros

Entrenamos al segundo modelo, esta vez realizando un randomized search para obtener los mejores hiperparámetros.

In [22]:
xgb = XGBRegressor()

param_grid = {
    "max_depth": [4, 8, 12],
    "learning_rate": [0.03, 0.05, 0.07, 1],
    "min_child_weight": [2, 4, 6, 8],
    "n_estimators": [250, 500, 750],
    "n_estimators": [250, 500, 750],
}

xgb_randcv = RandomizedSearchCV(
    xgb, param_grid, scoring="r2", cv=3, random_state=SEMILLA, n_iter=5
)
xgb_randcv.fit(dataset_train, target_train)

[19:44:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:46:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:48:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:50:41] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:52:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:54:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:56:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:58:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

RandomizedSearchCV(cv=3, estimator=XGBRegressor(), n_iter=5,
                   param_distributions={'learning_rate': [0.03, 0.05, 0.07, 1],
                                        'max_depth': [4, 8, 12],
                                        'min_child_weight': [2, 4, 6, 8],
                                        'n_estimators': [250, 500, 750]},
                   random_state=1, scoring='r2')

In [23]:
print("Mejores hiperparámetros: ", xgb_randcv.best_params_)
print("Mejor métrica: ", xgb_randcv.best_score_)


Mejores hiperparámetros:  {'n_estimators': 750, 'min_child_weight': 4, 'max_depth': 8, 'learning_rate': 0.05}
Mejor métrica:  0.9060478050906294


Entrenamos al modelo con la mejor métrica.

In [24]:
xgb_model = xgb_randcv.best_estimator_
model = xgb_model.fit(X=dataset_train, y=target_train)

[20:17:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Analizamos su performance con los datos de train.

In [25]:
precios_predichos_xgb_train = model.predict(dataset_train)
imprimir_metricas_regresion(target_train, precios_predichos_xgb_train)

El error (mse) de test es: 359116789.4691835
El error (rmse) de test es: 18950.377027098526
El score R2 es: 0.9665192714168835


Nuevamente, vemos que performa demasiado bien, por lo que podría haber overfitting.

Analizamos cómo performa con los datos de test.

In [26]:
precios_predichos_xgb = model.predict(dataset_test)
imprimir_metricas_regresion(target_test, precios_predichos_xgb)


El error (mse) de test es: 992426184.337675
El error (rmse) de test es: 31502.796452659168
El score R2 es: 0.9065789931399353


Sigue teniendo muy buenos resultados, similares a los obtenidos con el modelo anterior. 

## Conclusiones

El dataset con los aspectos tiene una mejor performance que sin, pero la mejora tampoco es extrema.